In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Installing the required packages and setting up the environment
!pip install llama_index.llms.openai
!pip install llama-index-embeddings-huggingface
!pip install openai
!pip install trulens.core
!pip install trulens.providers.openai
!pip install trulens.apps.llamaindex
!pip install llama-index.core
#!pip install llama-index

In [ ]:
from trulens.core import Feedback, Select
from trulens.providers.openai import OpenAI as TruOpenAI
from google.colab import userdata
import os
import openai
import numpy as np

openai.api_key=userdata.get('OPENAI_API_KEY');

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/core/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
from llama_index.core import VectorStoreIndex, Settings
from llama_index.llms.openai import OpenAI
from trulens.providers.openai import OpenAI as TruOpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
provider = TruOpenAI(api_key=openai.api_key)

Settings.llm = llm
Settings.embed_model = "local:BAAI/bge-small-en-v1.5"

In [ ]:
#!pip install llama-index-readers-file
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=["/content/acad_55.pdf"]).load_data()

In [ ]:
from llama_index.core import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [ ]:
from llama_index.core.node_parser import HierarchicalNodeParser

# create the hierarchical node parser w/ default settings
node_parser = HierarchicalNodeParser.from_defaults(
    chunk_sizes=[2048, 512, 128]
)

In [ ]:
nodes = node_parser.get_nodes_from_documents([document])
len(nodes)

714

In [ ]:
from llama_index.core.node_parser import get_leaf_nodes, get_root_nodes
leaf_nodes = get_leaf_nodes(nodes)

In [ ]:
len(leaf_nodes)

559

In [ ]:
nodes_by_id = {node.node_id: node for node in nodes}
parent_node = nodes_by_id[leaf_nodes[30].parent_node.node_id]
#parent_node = get_root_nodes(nodes)
#print(len(parent_node))

In [ ]:
print(len(parent_node.text))

2091


In [ ]:
import os

from llama_index.core import (
    ServiceContext,
    StorageContext,
    VectorStoreIndex,
    load_index_from_storage,
)

from llama_index.core import StorageContext, load_index_from_storage
from llama_index.core.retrievers import AutoMergingRetriever
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.core.query_engine import RetrieverQueryEngine

def build_automerging_index(
    documents, llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index", chunk_sizes=None,
):
    chunk_sizes = chunk_sizes or [2048, 512, 128]
    node_parser=HierarchicalNodeParser.from_defaults(chunk_sizes=chunk_sizes)
    nodes=node_parser.get_nodes_from_documents(documents)
    leaf_nodes = get_leaf_nodes(nodes)
    merging_context = ServiceContext.from_defaults(
        llm=Settings.llm, embed_model=Settings.embed_model,
    )
    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    if not os.path.exists(save_dir):
        automerging_index = VectorStoreIndex(
            leaf_nodes, storage_context=storage_context,
            service_context=merging_context)
        automerging_index.storage_context.persist(persist_dir=save_dir)
    else:
        automerging_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=save_dir),
            service_context=merging_context,
        )
    return automerging_index

def get_automerging_query_engine(
    automerging_index,s_top_k=5, rerank_top_n=1,):
    base_retriever=automerging_index.as_retriever(similarity_top_k=s_top_k)
    retriever = AutoMergingRetriever(base_retriever,
        automerging_index.storage_context, verbose=True
    )
    rerank = SentenceTransformerRerank(
        top_n=rerank_top_n, model="BAAI/bge-reranker-base"
    )
    auto_merging_engine=RetrieverQueryEngine.from_args(
        retriever, node_postprocessors=[rerank]
    )
    return auto_merging_engine

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Settings

index = build_automerging_index(
    [document],
    llm=llm,
    save_dir="./merging_index",
)

<ipython-input-15-73fa72dfef1c>:26: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  merging_context = ServiceContext.from_defaults(


In [ ]:
#query_engine = get_automerging_query_engine(index, similarity_top_k=3)

In [ ]:
from trulens.core import TruSession

session = TruSession()
session.reset_database()

🦑 TruSession initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of `TruSession` to prevent this.


In [ ]:
auto_merging_index = build_automerging_index(
    documents,
    llm=Settings.llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="./merging_index",
    chunk_sizes=[2048,512,128],
)

<ipython-input-15-73fa72dfef1c>:26: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  merging_context = ServiceContext.from_defaults(


In [ ]:
auto_merging_engine = get_automerging_query_engine(
    auto_merging_index,
    similarity_top_k=5,
    rerank_top_n=1,
)

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

In [ ]:
from trulens.apps.llamaindex import TruLlama
from trulens.core import Feedback, Select
#from trulens.feedback import GroundTruthAgreement

context_selection = TruLlama.select_source_nodes().node.text

f_qa_relevance = Feedback(
    provider.relevance,  name="Answer Relevance"
).on_input_output()

f_qs_relevance = (
    Feedback(provider.context_relevance, name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

f_groundedness = (
        Feedback(provider.groundedness_measure_with_cot_reasons,name="Groundedness" )
    .on(context_selection)
    .on_output()
)


[nltk_data] Downloading package stopwords to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     /usr/local/lib/python3.10/dist-
[nltk_data]     packages/llama_index/legacy/_static/nltk_cache...
[nltk_data]   Unzipping tokenizers/punkt.zip.


✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input context will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [ ]:
tru_recorder = TruLlama(auto_merging_engine,app_id="Auto-merging", app_version="v2", feedbacks=[f_qa_relevance,f_qs_relevance,f_groundedness])

In [ ]:
my_questions = []
with open('iith_questions_50.txt', 'r') as file:
    for line in file:
        item = line.strip()
        my_questions.append(item)

In [ ]:
print(len(my_questions))

50


In [ ]:
for question in my_questions:
  with tru_recorder as recording:
    response = auto_merging_engine.query(question)
    print(response)

There is no CGPA criteria for pursuing the Semester Internship.
The one time course conversion limit for 2023 BTech students of all departments is within the maximum permissible time of six years.
No, there is no mention of any stipend for BTech students working as Teaching Assistants in the provided context information.
The maximum permissible time for BTech students to complete all the credit requirements is six years.
The minimum number of extra credits required to earn a minor degree is 12.
> Merging 3 nodes into parent node.
> Parent node id: a716f6e0-b9eb-486c-9d98-ef3014d2b8ae.
> Parent node text: ● If a student drops from the Honors program, they can convert the earned 3 credits into free or ...

A student cannot do a double major and minor in the same department according to the provided context information.
The application deadline for conversion from B.Tech to B.Tech+Ph.D. is the last day of instruction of the 6th semester.
Internship during vacation period is not a mandator

In [ ]:
session.get_leaderboard(app_ids=[])

,,Context Relevance,Answer Relevance,Groundedness,latency,total_cost
app_name,app_version,,,,,
Auto-merging,v2,0.773333,0.802,0.7712,6.44,0.0


In [ ]:
records, feedback= session.get_records_and_feedback(app_ids=[])


In [ ]:
import pandas as pd

rag_feedback = pd.DataFrame(records)

rag_feedback.to_excel (r'rag_auto3merge_10-09-2024.xlsx', index = None, header=True)
print('DataFrame is written to Excel File successfully.')

DataFrame is written to Excel File successfully.


In [ ]:
!pip install trulens.dashboard

  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 520.5/520.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 59.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 706.3/706.3 kB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 63.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.2/75.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 680.8/680.8 kB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.4/635.4 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 47.6 MB

In [ ]:
from trulens.dashboard import run_dashboard
run_dashboard(session)

Starting dashboard ...
npm WARN exec The following package was not found and will be installed: localtunnel@2.0.2

Go to this url and submit the ip given here. your url is: https://neat-spoons-join.loca.lt

  Submit this IP Address: 34.138.19.205


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>